## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../content/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
cleaned_app_df = application_df.drop(["EIN","NAME"], axis= "columns")

In [3]:
cleaned_app_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Determine the number of unique values in each column.
cleaned_app_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
cleaned_app_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
bins = cleaned_app_df.groupby("APPLICATION_TYPE", as_index=False).count()
bins

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,528,528,528,528,528,528,528,528,528
1,T12,27,27,27,27,27,27,27,27,27
2,T13,66,66,66,66,66,66,66,66,66
3,T14,3,3,3,3,3,3,3,3,3
4,T15,2,2,2,2,2,2,2,2,2
5,T17,1,1,1,1,1,1,1,1,1
6,T19,1065,1065,1065,1065,1065,1065,1065,1065,1065
7,T2,16,16,16,16,16,16,16,16,16
8,T25,3,3,3,3,3,3,3,3,3
9,T29,2,2,2,2,2,2,2,2,2


In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
bins = cleaned_app_df.groupby("APPLICATION_TYPE", as_index=False).count()
application_types_to_replace = bins[bins.AFFILIATION <528]["APPLICATION_TYPE"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
class_df = cleaned_app_df["CLASSIFICATION"].value_counts()
class_df

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_bins = class_df[class_df>1]
class_bins

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_df[class_df<1882].index)
classifications_to_replace
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
cleaned_app_df = pd.get_dummies(cleaned_app_df,dtype=float)
cleaned_app_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
X= cleaned_app_df.drop("IS_SUCCESSFUL",axis=1).values
y= cleaned_app_df["IS_SUCCESSFUL"]


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# nn = tf.keras.models.Sequential()

# # First hidden layer

# nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=116))
# # Second hidden layer

# nn.add(tf.keras.layers.Dense(units=30, activation="relu"))
# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# # Check the structure of the model
# nn.summary()

In [17]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# nn = tf.keras.models.Sequential()

# # First hidden layer

# nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=116))
# # Second hidden layer

# nn.add(tf.keras.layers.Dense(units=50, activation="relu"))
# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# # Check the structure of the model
# nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 100)               11700     
                                                                 
 dense_4 (Dense)             (None, 50)                5050      
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 16,801
Trainable params: 16,801
Non-trainable params: 0
_________________________________________________________________


In [24]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# nn = tf.keras.models.Sequential()

# # First hidden layer

# nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=116))
# # Second hidden layer

# nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# # Third hidden layer

# nn.add(tf.keras.layers.Dense(units=15, activation= "relu"))
# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# # Check the structure of the model
# nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                9360      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 15)                465       
                                                                 
 dense_9 (Dense)             (None, 1)                 16        
                                                                 
Total params: 12,271
Trainable params: 12,271
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=116))
# Second hidden layer

nn.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Third hidden layer

nn.add(tf.keras.layers.Dense(units=25, activation= "relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 100)               11700     
                                                                 
 dense_11 (Dense)            (None, 50)                5050      
                                                                 
 dense_12 (Dense)            (None, 25)                1275      
                                                                 
 dense_13 (Dense)            (None, 1)                 26        
                                                                 
Total params: 18,051
Trainable params: 18,051
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5722 - accuracy: 0.7178
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7304
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5466 - accuracy: 0.7332
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5455 - accuracy: 0.7342
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5443 - accuracy: 0.7339
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5432 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5433 - accuracy: 0.7352
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5420 - accuracy: 0.7360
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5411 - accura

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5753 - accuracy: 0.7324 - 476ms/epoch - 2ms/step
Loss: 0.5752664804458618, Accuracy: 0.7323614954948425


In [ ]:
# Export our model to HDF5 file
nn.save("content/AlphabetSoupCharity_Optimization.h5(1)")

from google.colab import files
files.download("content/AlphabetSoupCharity_Optimization.h5(1)")

The purpose of this google colab notebook is to use machine learning and neural networks to form a classifier to predict who is going to be successful and who should receive funding.

### Data Processing
- The variables used in this model are classification and application type.

- The features used are 

    AFFILIATION—Affiliated sector of industry

    USE_CASE—Use case for funding

    ORGANIZATION—Organization type

    STATUS—Active status

    INCOME_AMT—Income classification

    SPECIAL_CONSIDERATIONS—Special consideration for application

    ASK_AMT—Funding amount requested

- Before using this model the IS_SUCCESSFUL column should be removed.

### Compiling, Training, and Evaluating the Model

- 111 neurons, 3 layers, and 2 activation methods were used
- 151 neurons, 3 layers, and 2 acitvation methods were used
- 126 neurons, 4 layers, and 2 activation methods were used
- 176 neurons, 4 layers, and 2 activation methods were used
- The model was able to achieve 73% accuracy not the goal of 75%
- Increasing and modifying the hidden layers and neuron were used to increase the model preformance.

The model still has room to be improved but has 73% accuracy rating to start meaning the results are usally in your favor of predicting it right but not something you necessary want to trust without taking further steps to get a high accuracy score.

Trying a different activation method or adding more hidden layers is a great place to start for improving this model as adding another hidden layer when building this improved the score by roughly 2 points. 

